In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import filteranddeltaF as dFF_func
import plotdeltaF_angles as df_plot
import meanByangle as df_angle_mean

from scipy.stats import ttest_ind, ttest_rel, zscore
from roi_matching import process_matched_neurons, convert_mat_to_csv

import alignment_code as df_align

import osi_calculation as osi
import gosiAnalysis as gosi
import correlationori as corr
import filtercriteria as filter_criteria
import plotly.graph_objects as go
import helper_functions as hf
import helper_functions
import snr_calculation as snr
import cross_correlation as cc

import warnings


In [ ]:
ml = "vgatnpy"
pre = "pre"
post= "post"

In [ ]:
#load the pre and post session data
pre_df = pd.read_csv(rf'pre\dFF_{ml}_pre.csv', delimiter=",", header=0, decimal='.',engine='python')
post_df = pd.read_csv(rf'post\dFF_{ml}_post.csv', delimiter=",", header=0, decimal='.',engine='python')

In [ ]:
is_cell_pre_path = rf'J:\My Drive\0-Main\1_STRESS\{ml}\pre\iscell.npy'
is_cell_post_path = rf'J:\My Drive\0-Main\1_STRESS\{ml}\post\iscell.npy'
mat_file_path = rf'J:\My Drive\0-Main\1_STRESS\{ml}\matched_rois.csv'

df_match, list_pre, list_post = process_matched_neurons(mat_file_path, is_cell_pre_path, is_cell_post_path)


#conmvert the floats to int in the list
list_pre = [int(i) for i in list_pre]
list_post = [int(i) for i in list_post]

list_pre=helper_functions.convert_list_to_int_str(list_pre)
list_post=helper_functions.convert_list_to_int_str(list_post)

#filter pre_df and post_df based on matched neurons
essen = helper_functions.get_essential_columns_as_list(pre_df)

#combine essen list with list_pre
list_pre.extend(essen)
list_post.extend(essen)

#filter the df column based on the list_pre and list_post
pre_df = pre_df[list_pre]
post_df = post_df[list_post]

pre_df_seq_raw = hf.resequence_digit_columns(pre_df)
post_df_seq_raw = hf.resequence_digit_columns(post_df)

#drop the NaN rows in both df if either pre or post has NaN

def drop_columns_with_nan_rows(df1, df2):
    # Find columns with NaN in corresponding rows
    mask = (df1.isna() | df2.isna()).any()
    
    # Drop those columns from both DataFrames
    df1_cleaned = df1.loc[:, ~mask]
    df2_cleaned = df2.loc[:, ~mask]
    
    return df1_cleaned, df2_cleaned


pre_df_seq, post_df_seq = drop_columns_with_nan_rows(pre_df_seq_raw, post_df_seq_raw)

pre_df_nonan , post_df_nonan = drop_columns_with_nan_rows(pre_df, post_df)

# #replace the NaN values with 0
# pre_df_seq.fillna(0, inplace=True)
# post_df_seq.fillna(0, inplace=True)

In [ ]:
#load the pre and post session data
pre_df = pd.read_csv(rf'pre\dFF_{ml}_pre.csv', delimiter=",", header=0, decimal='.',engine='python')
post_df = pd.read_csv(rf'post\dFF_{ml}_post.csv', delimiter=",", header=0, decimal='.',engine='python')

In [ ]:
df_align.plot_raster_plotly(pre_df_seq)

In [ ]:
df_align.plot_raster_plotly(post_df_seq)

In [ ]:
responsive_preneurons_df, plotly_fig = filter_criteria.criteria_plot_population_response(pre_df_seq, inclusion_criterion="IC5", z_threshold=1, secondsbefore=0.5, secondsafter=4, reorder_rois=False)


In [ ]:
responsive_postneurons_df, plotly_fig = filter_criteria.criteria_plot_population_response(post_df_seq[responsive_preneurons_df.columns], inclusion_criterion=None, z_threshold=1, secondsbefore=0.5, secondsafter=4, reorder_rois=False)


In [ ]:
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
filter_criteria.filter_criteria(pre_df_seq, inclusion_criterion=None, z_threshold=1, secondsbefore=1, secondsafter=6)  

In [ ]:
rois = hf.get_digit_columns(responsive_preneurons_df)

df_angle_mean.calculate_mean_by_angle(responsive_preneurons_df, output_path='meanAngle_pre.csv',Select_columns_toplot=rois)
pre_gosi_results = gosi.calculate_gosi("meanAngle_pre.csv", column_names=rois)


df_angle_mean.calculate_mean_by_angle(responsive_postneurons_df, output_path='meanAngle_post.csv',Select_columns_toplot=rois)
post_gosi_results = gosi.calculate_gosi("meanAngle_post.csv", column_names=rois)


In [ ]:
post_gosi_results.to_csv('post_gosi_results.csv', index=False)
pre_gosi_results.to_csv('pre_gosi_results.csv', index=False)

In [ ]:
gosi.plot_gosi('pre_gosi_results.csv', "post_gosi_results.csv")

In [ ]:
osi.calculate_and_plot(pre_gosi='pre_gosi_results.csv', mean_pre ='meanAngle_pre.csv',post_gosi ="post_gosi_results.csv", mean_post= "meanAngle_post.csv", osi_pre='osi_results_pre.csv',osi_post='osi_results_post.csv')

In [ ]:
snr.process_and_visualize_snr(pre_df_seq, post_df_seq)

In [ ]:
adjusted_df, fit_results = gosi.process_and_plot_gosi_curve('meanAngle_post.csv', 'post_gosi_results.csv')


In [ ]:
adjusted_df, fit_results = gosi.process_and_plot_gosi_curve('meanAngle_pre.csv', 'pre_gosi_results.csv')


In [ ]:
cc.analyze_correlations(pre_df, post_df, name_of_v="Pupil size")

In [ ]:
cc.analyze_correlations(pre_df, post_df, correlate_with='speed', name_of_v='Locomotion')

In [ ]:
cc.plot_cross_correlations(pre_df)
cc.plot_cross_correlations(post_df)

In [ ]:
cc.plot_cross_correlations(pre_df, correlate_with="speed")
cc.plot_cross_correlations(post_df, correlate_with="speed")

In [ ]:
cc.plot_roi_correlation_matrix(pre_df)
cc.plot_roi_correlation_matrix(post_df)

In [ ]:
def barplot_dff(df1, df2):
    col_df1 = helper_functions.get_digit_columns(df1)
    col_df2 = helper_functions.get_digit_columns(df2)

    df1 = df1[col_df1]
    df2 = df2[col_df2]

    #takes the mean of each column
    df1_mean_col = df1.mean()
    df2_mean_col = df2.mean()

    #take the mean across columns
    df1_mean = df1.mean().mean()
    df2_mean = df2.mean().mean()

    #get the ci of the df1 and df2
    df1_ci = df1.mean().sem()
    df2_ci = df2.mean().sem()


    print(df1_mean, df2_mean)


    #plot the mean of the two df inone plot with each bar beingfrom df
    sns.set(style="whitegrid")
    fig, ax = plt.subplots(figsize=(4, 7))
    #set figure size
    sns.barplot(x=["Pre-stress", "Post-stress"], y=[df1_mean, df2_mean], , estimator='mean', yerr=[df1_ci, df2_ci], width = 0.5, capsize=0.1, ax=ax)
    #plot the indivdual data points from df1_mean_col and df2_mean_col
    sns.swarmplot(data=[df1_mean_col, df2_mean_col], color="0.2", size=2)

    plt.show()

barplot_dff(pre_df_seq, post_df_seq)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

def barplot_dff(df1, df2):
    col_df1 = helper_functions.get_digit_columns(df1)
    col_df2 = helper_functions.get_digit_columns(df2)

    df1 = df1[col_df1]
    df2 = df2[col_df2]

    # Takes the mean of each column
    df1_mean_col = df1.mean()
    df2_mean_col = df2.mean()

    # Take the mean across columns
    df1_mean = df1.mean().mean()
    df2_mean = df2.mean().mean()

    # Get the CI of the df1 and df2
    df1_ci = df1.sem().mean()
    df2_ci = df2.sem().mean()

    print(df1_mean, df2_mean)

    # Styling to make it more Prism-like
    sns.set(style="ticks", palette="pastel")

    # Create the plot
    fig, ax = plt.subplots(figsize=(4, 6))
    bars = sns.barplot(x=["Pre-stress", "Post-stress"], y=[df1_mean, df2_mean], yerr=[df1_ci, df2_ci], 
                        width=0.5, capsize=0.1, ax=ax, edgecolor="black", linewidth=1.5, errcolor='gray', errwidth=1.5)

    # Adding the individual points
    mean_data = pd.DataFrame({'Condition': ['Pre-stress']*len(df1_mean_col) + ['Post-stress']*len(df2_mean_col),
                              'Value': pd.concat([df1_mean_col, df2_mean_col], ignore_index=True)})
    sns.swarmplot(x="Condition", y="Value", data=mean_data, color="0.25", size=4, ax=ax, dodge=True)

    # Enhancements for a Prism-like appearance
    ax.set_ylabel("Mean DF/F%")
    ax.set_xlabel("")
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.grid(True, which='major', axis='y', linestyle='--', linewidth=0.5)
    ax.tick_params(axis='y', which='major', labelsize=10)
    ax.tick_params(axis='x', which='major', labelsize=10)

    plt.tight_layout()
    plt.show()

# Assuming pre_df_seq and post_df_seq are defined elsewhere
barplot_dff(pre_df_seq, post_df_seq)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import helper_functions
from scipy import stats

def barplot_dff(file_pairs):
    all_data = []
    conditions = []
    mice = []

    for i, (df1, df2) in enumerate(file_pairs):
        try:
            col_df1 = helper_functions.get_digit_columns(df1)
            col_df2 = helper_functions.get_digit_columns(df2)

            df1 = df1[col_df1]
            df2 = df2[col_df2]

            # Store data
            all_data.extend([df1.values.flatten(), df2.values.flatten()])
            conditions.extend(['Pre-stress'] * df1.size + ['Post-stress'] * df2.size)
            mice.extend([f'Mouse {i+1}'] * (df1.size + df2.size))
        except Exception as e:
            print(f"Error processing data for Mouse {i+1}: {str(e)}")
            continue

    if not all_data:
        print("No valid data to process.")
        return None, None

    # Create DataFrame for all data
    combined_data = pd.DataFrame({
        'Condition': conditions,
        'Value': np.concatenate(all_data),
        'Mouse': mice
    })

    # Perform t-test
    pre_stress = combined_data[combined_data['Condition'] == 'Pre-stress']['Value']
    post_stress = combined_data[combined_data['Condition'] == 'Post-stress']['Value']
    t_statistic, p_value = stats.ttest_ind(pre_stress, post_stress)

    # Calculate means and standard errors
    summary_data = combined_data.groupby('Condition').agg({
        'Value': ['mean', 'sem']
    }).reset_index()
    summary_data.columns = ['Condition', 'Mean', 'SEM']

    # Plotting
    sns.set(style="ticks", palette="pastel")
    fig, ax = plt.subplots(figsize=(6, 8))

    # Bar plot
    sns.barplot(x='Condition', y='Mean', data=summary_data,
                capsize=0.1, ax=ax, edgecolor="black", linewidth=1.5, errcolor='gray', errwidth=1.5)

    # Swarm plot
    sns.swarmplot(x="Condition", y="Value", data=combined_data, 
                  dodge=True, color="0.25", size=2, ax=ax)

    # Styling
    ax.set_ylabel("Mean DF/F%")
    ax.set_xlabel("")
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.grid(True, which='major', axis='y', linestyle='--', linewidth=0.5)
    ax.tick_params(axis='y', which='major', labelsize=10)
    ax.tick_params(axis='x', which='major', labelsize=10)

    # Add p-value annotation
    y_max = combined_data['Value'].max()
    ax.text(0.5, y_max*1.1, f'p = {p_value:.4f}', ha='center', va='bottom')

    plt.tight_layout()
    plt.show()

    print(f"T-statistic: {t_statistic}")
    print(f"P-value: {p_value}")

    return combined_data, summary_data

In [ ]:
df_wt1_pre = pd.read_csv(rf'J:\My Drive\0-Main\1_STRESS\wt1\pre\dFF_wt1_pre.csv', delimiter=",", header=0, decimal='.',engine='python')
df_wt1_post = pd.read_csv(rf'J:\My Drive\0-Main\1_STRESS\wt1\post\dFF_wt1_post.csv', delimiter=",", header=0, decimal='.',engine='python')
df_wt2_pre = pd.read_csv(rf'J:\My Drive\0-Main\1_STRESS\wt2\pre\dFF_wt2_pre.csv', delimiter=",", header=0, decimal='.',engine='python')
df_wt2_post = pd.read_csv(rf'J:\My Drive\0-Main\1_STRESS\wt2\post\dFF_wt2_post.csv', delimiter=",", header=0, decimal='.',engine='python')
df_wt3_pre = pd.read_csv(rf'J:\My Drive\0-Main\1_STRESS\wt3\pre\dFF_wt3_pre.csv', delimiter=",", header=0, decimal='.',engine='python')
df_wt3_post = pd.read_csv(rf'J:\My Drive\0-Main\1_STRESS\wt3\post\dFF_wt3_post.csv', delimiter=",", header=0, decimal='.',engine='python')

file_pairs = [(df_wt1_pre, df_wt1_post), (df_wt2_pre, df_wt2_post), (df_wt3_pre, df_wt3_post)]

barplot_dff(file_pairs)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import helper_functions
from scipy import stats

def barplot_dff(file_pairs):
    all_data = []
    conditions = []
    mice = []

    for i, (df1, df2) in enumerate(file_pairs):
        col_df1 = helper_functions.get_digit_columns(df1)
        col_df2 = helper_functions.get_digit_columns(df2)

        df1 = df1[col_df1]
        df2 = df2[col_df2]

        # Store data
        all_data.extend([df1.values.flatten(), df2.values.flatten()])
        conditions.extend(['Pre-stress'] * df1.size + ['Post-stress'] * df2.size)
        mice.extend([f'Mouse {i+1}'] * (df1.size + df2.size))

    # Create DataFrame for all data
    combined_data = pd.DataFrame({
        'Condition': conditions,
        'Value': np.concatenate(all_data),
        'Mouse': mice
    })

    # Perform nested t-test
    t_statistic, p_value = stats.ttest_ind(
        combined_data[combined_data['Condition'] == 'Pre-stress']['Value'],
        combined_data[combined_data['Condition'] == 'Post-stress']['Value']
    )

    # Calculate means and standard errors
    summary_data = combined_data.groupby('Condition').agg({
        'Value': ['mean', 'sem']
    }).reset_index()
    summary_data.columns = ['Condition', 'Mean', 'SEM']

    # Styling
    sns.set(style="ticks", palette="pastel")

    # Create the plot
    fig, ax = plt.subplots(figsize=(6, 8))
    bars = sns.barplot(x='Condition', y='Mean', data=summary_data,
                       capsize=0.1, ax=ax, edgecolor="black", linewidth=1.5, errcolor='gray', errwidth=1.5)

    # Adding the individual points
    sns.swarmplot(x="Condition", y="Value", data=combined_data, 
                  dodge=True, color="0.25", size=2, ax=ax)

    # Enhancements for a Prism-like appearance
    ax.set_ylabel("Mean DF/F%")
    ax.set_xlabel("")
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.grid(True, which='major', axis='y', linestyle='--', linewidth=0.5)
    ax.tick_params(axis='y', which='major', labelsize=10)
    ax.tick_params(axis='x', which='major', labelsize=10)

    # Add p-value annotation
    y_max = combined_data['Value'].max()
    ax.text(0.5, y_max*1.1, f'p = {p_value:.4f}', ha='center', va='bottom')

    plt.tight_layout()
    plt.show()

    print(f"T-statistic: {t_statistic}")
    print(f"P-value: {p_value}")

    return combined_data, summary_data




df_wt1_pre = pd.read_csv(rf'J:\My Drive\0-Main\1_STRESS\wt1\pre\dFF_wt1_pre.csv', delimiter=",", header=0, decimal='.',engine='python')
df_wt1_post = pd.read_csv(rf'J:\My Drive\0-Main\1_STRESS\wt1\post\dFF_wt1_post.csv', delimiter=",", header=0, decimal='.',engine='python')
df_wt2_pre = pd.read_csv(rf'J:\My Drive\0-Main\1_STRESS\wt2\pre\dFF_wt2_pre.csv', delimiter=",", header=0, decimal='.',engine='python')
df_wt2_post = pd.read_csv(rf'J:\My Drive\0-Main\1_STRESS\wt2\post\dFF_wt2_post.csv', delimiter=",", header=0, decimal='.',engine='python')
df_wt3_pre = pd.read_csv(rf'J:\My Drive\0-Main\1_STRESS\wt3\pre\dFF_wt3_pre.csv', delimiter=",", header=0, decimal='.',engine='python')
df_wt3_post = pd.read_csv(rf'J:\My Drive\0-Main\1_STRESS\wt3\post\dFF_wt3_post.csv', delimiter=",", header=0, decimal='.',engine='python')

file_pairs = [(df_wt1_pre, df_wt1_post), (df_wt2_pre, df_wt2_post), (df_wt3_pre, df_wt3_post)]

barplot_dff(file_pairs)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data
labels = ['Crh', 'Pmch', 'Hcrt', 'Gal']
values = [121, 0, 0, 0]

# Colors (pastel)
colors = ['#FFB3BA', '#BAFFC9', '#BAE1FF', '#FFFFBA']

# Create the plot
plt.figure(figsize=(4, 6))
bars = plt.bar(labels, values, color=colors, edgecolor='black', linewidth=1.5, width=0.7)

# Customize the plot
plt.title('PVN RNA Probe Count', fontsize=16, fontweight='bold')
plt.ylabel('Count', fontsize=12)
plt.ylim(0, 140)

# Add value labels on top of each bar
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height}',
             ha='center', va='bottom')

# Customize grid
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Customize spines
for spine in plt.gca().spines.values():
    spine.set_visible(False)

# Remove top and right spines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Thicken bottom and left spines
plt.gca().spines['bottom'].set_linewidth(1.5)
plt.gca().spines['left'].set_linewidth(1.5)

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
# Extract the second row from each mean column
second_row_df1 = [df1_mean_col.iloc[1]]
second_row_df2 = [df2_mean_col.iloc[1]]

# Plot using seaborn's swarmplot
sns.swarmplot(data=[second_row_df1, second_row_df2], color="0.2", size=2)